Function to replace all the token that appears less than k times with a single token

In [67]:
def clean_data(tokens_list = [], k = 1, special_token = '#UNK#'):
    token_freq = {}
    replaced_tokens = []
    for token in tokens_list:
        if token[0] not in token_freq: 
            replaced_tokens.append(token[0])
            token_freq[token[0]] = 1
        else:
            token_freq[token[0]] += 1
    for i in range(len(tokens_list)):
        if token_freq[tokens_list[i][0]] < k:
            tokens_list[i][0] = special_token
    
    return tokens_list, replaced_tokens

In [68]:
# Example usage
data = [['a', 'O'], ['b', 'O'], ['a', 'I'], ['c', 'O']]
clean_data(tokens_list=data, k = 2)

([['a', 'O'], ['#UNK#', 'O'], ['a', 'I'], ['#UNK#', 'O']], ['a', 'b', 'c'])

Function estimating the emission params

In [69]:
def emission(tokens_list = [], y = 'O', x = '', minimum_freq = 1):
    y_count = 0
    x_given_y_count = 0
    tokens_list, replaced_tokens = clean_data(tokens_list, k = minimum_freq)
    if x in replaced_tokens: 
        x = '#UNK#'
    for token in tokens_list:
        if token[1] == y:
            y_count += 1
            if token[0] == x:
                x_given_y_count += 1
                
    return float(x_given_y_count) / y_count

In [70]:
# Example ussage
import os
# data = [['a', 'O'], ['b', 'O'], ['a', 'I'], ['c', 'O']]
# emission(tokensList=data, y='O', x='a')
data = []

with open('./EN/train') as train_file:
    read_data = train_file.read()
    read_data = os.linesep.join([s for s in read_data.splitlines() if s])
    data = list(map(lambda x: x.split(' '),read_data.split('\n')))

emission(tokens_list=data, y='O', x='sake', minimum_freq = 5)

0.16372411517201552